# Playoffs Analysis

This notebook compines week_predictions.ipynb with player_analysis.ipynb to the two playof weeks to allow for detailed analysis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

## Player Analysis

In [ ]:
import sys
sys.path.append("..")
from base_python_files.api_info import *
from base_python_files.credentials import *
from base_python_files.api_functions import *
from base_python_files.stat_functions import *

In [ ]:
d = retrieve_espn_data(swid,espn_s2,league_id)
data = clean_data(d)

In [ ]:

scoring_weights = {
    "FOW": 1,
    "assists": 1,
    "average_time_on_ice": 1,
    "blocks": 0,
    "goals": 1,
    "hits": 1,
    "penalty_minutes": 1,
    "plus_minus": 0,
    "power play points": 1,
    "shots_on_goal": 1,
}


scoring_weights = {
    "FOW": 1,
    "assists": 0,
    "average_time_on_ice": 1,
    "blocks": 0,
    "goals": 0,
    "hits": 1,
    "penalty_minutes": 0,
    "plus_minus": 0,
    "power play points": 1,
    "shots_on_goal": 1,
}

In [ ]:
games = pd.read_csv("../schedules/playoff_week_2_games.csv")
away_team_number = dict(games["Away Team"].value_counts())
home_team_number = dict(games["Home Team"].value_counts())
team_total_plays = {}
for team in team_dict.values():
    if team not in away_team_number:
        away_team_number[team] = 0
    if team not in home_team_number:
        home_team_number[team] = 0
    team_total_plays[team] = int(away_team_number[team] + home_team_number[team])
team_total_plays[np.nan] = 0


weekend_games = games[games["Day"].isin(["Saturday", "Sunday"])]
away_team_number = dict(weekend_games["Away Team"].value_counts())
home_team_number = dict(weekend_games["Home Team"].value_counts())
team_weekend_plays = {}
for team in team_dict.values():
    if team not in away_team_number:
        away_team_number[team] = 0
    if team not in home_team_number:
        home_team_number[team] = 0
    team_weekend_plays[team] = int(away_team_number[team] + home_team_number[team])
team_weekend_plays[np.nan] = 0


skater_data = data[(data["Position"] != "GK")].copy()

played_players = skater_data  # [skater_data['games_played so far']>=3]
played_players = skater_data[(skater_data["games_playedAT"] >= 2)]
for addition in [" week", " weekend", " single"]:
    for stat_name in all_stats_names:
        played_players["sum scores " + stat_name + addition] = 0

played_players["Team"] = played_players["TeamID"].map(team_dict)
played_players["Games in playoff"] = played_players["Team"].apply(
    lambda x: team_total_plays[x]
)
played_players["Games in playoff weekend"] = played_players["Team"].apply(
    lambda x: team_weekend_plays[x]
)

for cat in scoring_weights:
    for stat_name in all_stats_names:
        played_players[cat + stat_name + " week"] = (
            played_players[cat + stat_name] * played_players["Games in playoff"]
        )
        played_players[cat + stat_name + " weekend"] = (
            played_players[cat + stat_name]
            * played_players["Games in playoff weekend"]
        )
        played_players[cat + stat_name + " single"] = played_players[cat + stat_name]
        
    # SHOULD ATOI BE INCLUDED IN THIS?

In [ ]:
for cat in scoring_weights:
    for stat_name in all_stats_names:
        # cat_team_mean = played_players[~played_players["LeagueTeamID"].isna()][cat+stat_name].mean()
        # cat_team_std = played_players[~played_players["LeagueTeamID"].isna()][cat+stat_name].std()
        cat_team_mean = played_players[played_players["LeagueTeamID"] == 3][
            cat + "AT"
        ].mean()
        cat_team_std = played_players[played_players["LeagueTeamID"] == 3][
            cat + "AT"
        ].std()
        for addition in [" week", " weekend"," single"]:
            played_players[cat + " stat" + addition] = (
                played_players[cat + stat_name + addition] - cat_team_mean
            ) / cat_team_std
            # played_players[cat+" stat"]=(played_players[cat+ " so far"]-played_players[cat+ " so far"].mean())/played_players[cat+ " so far"].std()
            if cat in scoring_weights:
                played_players["sum scores " + stat_name + addition] += (
                    played_players[cat + " stat" + addition] * scoring_weights[cat]
                )

played_players = played_players[
    ["ID", "Name", "TeamID", "LeagueTeamID", "Position", "InjuryStatus"]
    + ["sum scores " + stat_name + " week" for stat_name in all_stats_names]
    + ["sum scores " + stat_name + " weekend" for stat_name in all_stats_names]
    + ["sum scores " + stat_name + " single" for stat_name in all_stats_names]
    + ["games_played" + stat_name for stat_name in all_stats_names]
]
# played_players=played_players[played_players["InjuryStatus"]!='INJURY_RESERVE']

my_players = played_players[(played_players["LeagueTeamID"] == 8)]
available_players = played_players[
    (played_players["InjuryStatus"] == "ACTIVE")
    & (played_players["LeagueTeamID"].isna())
]

In [ ]:
teams_not_playing_saturday = [
    "Anaheim Ducks",
    "Pittsburgh Penguins",
    "Columbus Blue Jackets",
    "Winnipeg Jets",
]

reversed_team_ids = {v: k for k, v in team_dict.items()}
team_ids_not_playing_saturday = [
    reversed_team_ids[team] for team in teams_not_playing_saturday
]

In [ ]:
played_players[
    (played_players["Position"] == "D")
    & (played_players["TeamID"].isin(team_ids_not_playing_saturday))
].sort_values(by="sum scores AT week", ascending=False).iloc[0:10]

In [ ]:
played_players[
    (played_players["TeamID"].isin(team_ids_not_playing_saturday))
].sort_values(by="sum scores AT week", ascending=False).iloc[0:10]

In [ ]:

played_players[
    (played_players["LeagueTeamID"].isna())
    & (played_players["Position"] == "D")
    & (played_players["TeamID"].isin(team_ids_not_playing_saturday))
].sort_values(by="sum scores AT week", ascending=False).iloc[0:10]

In [ ]:
teams_playing_all_weekend = [
    "Boston Bruins",
    "Buffalo Sabres",
    "Carolina Hurricanes",
    "Chicago Blackhawks",
    "Florida Panthers",
    "Los Angeles Kings",
    "Nashville Predators",
    "Philadelphia Flyers",
    "St. Louis Blues",
    "Tampa Bay Lightning",
    "Vegas Golden Knights",
]
reversed_team_ids = {v: k for k, v in team_dict.items()}
team_ids_playing_all_weekend = [
    reversed_team_ids[team] for team in teams_playing_all_weekend
]
played_players[
    (played_players["LeagueTeamID"].isna())
    & (played_players["TeamID"].isin(team_ids_playing_all_weekend))
].sort_values(by="sum scores AT weekend", ascending=False).iloc[0:10]

In [ ]:
played_players[
    (played_players["LeagueTeamID"].isna())
    & (played_players["Position"].str.contains("D"))
].sort_values(by="sum scores AT weekend", ascending=False)[["Name","Position","InjuryStatus","sum scores 15 weekend","sum scores 30 weekend","sum scores AT weekend"]].iloc[0:10]

In [ ]:
played_players.columns

In [ ]:
played_players[
    (played_players["LeagueTeamID"].isna())
].sort_values(by="sum scores AT week", ascending=False)[["Name","TeamID","Position","InjuryStatus","sum scores 15 week","sum scores 30 week","sum scores AT week"]].iloc[0:20]

In [ ]:
played_players[played_players["Name"]=="Juraj Slafkovsky"][["Name","Position","InjuryStatus","sum scores 15 single","sum scores 30 single","sum scores AT single"]]

In [ ]:
played_players[(played_players["LeagueTeamID"].isna())].sort_values(
    by="sum scores 30 single", ascending=False
)[["Name","Position","InjuryStatus","sum scores 15 single","sum scores 30 single","sum scores AT single"]].iloc[0:20]

In [ ]:
my_players.sort_values(by="sum scores 15 week", ascending=True)

## Simulation

In [ ]:
skater_cats = [
    "goals",
    "assists",
    "plus_minus",
    "penalty_minutes",
    "power play points",
    "FOW",
    "average_time_on_ice",
    "shots_on_goal",
    "hits",
    "blocks",
]

In [ ]:
metric = "30"

skater_data = data[(data["Position"] != "GK")]

played_players = skater_data.copy()

played_players["Team"] = played_players["TeamID"].map(team_dict)

for cat in skater_cats:
    played_players[cat] = played_players[cat + metric]
played_players = played_players[["ID", "Name", "Team", "LeagueTeamID"] + skater_cats]

In [ ]:
games = pd.read_csv("../schedules/playoff_week_2_games.csv")
# games = pd.read_csv("schedules/penultimate_week.csv")

opp_team_id = 3

g_team = played_players[played_players["LeagueTeamID"] == 8]
opp_team = played_players[played_players["LeagueTeamID"] == opp_team_id]

In [ ]:
len(g_team), len(opp_team)

In [ ]:
current_opp_scores_list = [0 for _ in skater_cats]
current_g_scores_list = [0 for _ in skater_cats]


current_g_atoi = 0
current_g_players = 1
current_opp_atoi = 0
current_opp_players = 1

current_opp_scores = {
    skater_cats[i]: current_opp_scores_list[i] for i in range(len(skater_cats))
}
current_g_scores = {
    skater_cats[i]: current_g_scores_list[i] for i in range(len(skater_cats))
}


dropped_days_g = {}
dropped_days_opp = {}
swapped_days_g = {}
swapped_days_opp = {}

In [ ]:
swapped_days_g["Monday"] = [("Sean Monahan",),("Ryan Donato",),("Dmitri Voronkov", "Mathieu Olivier")]
dropped_days_g["Thursday"]=["Boone Jenner"]
if opp_team_id == 1:
    swapped_days_opp["Monday"] = [("Phillip Danault",),('Charlie McAvoy',)]
    dropped_days_opp["Thursday"] = ["Ivan Barbashev", "Filip Hronek"]
    dropped_days_opp["Saturday"] = ["Ivan Barbashev", "Filip Hronek"]
    swapped_days_g["Saturday"] = [
    ("Adam Fantilli", "Ryan Donato"),
    ("Zach Werenski", "Morgan Geekie"),
    ("Boone Jenner","Erik Cernak")]
    swapped_days_g["Sunday"] = [("Vince Dunn","Jason Zucker")]
elif opp_team_id == 3:
    swapped_days_opp["Monday"] = [
        ("Tony DeAngelo",),
        ("Noah Dobson",),
        ("Dylan Holloway",),
    ]
    swapped_days_g["Saturday"] = [
    ("Adam Fantilli", "Ryan Donato"),
    ("Zach Werenski", "Morgan Geekie"),
    ("Boone Jenner","Niko Mikkola")]
    swapped_days_g["Sunday"] = [("Vince Dunn","Jason Zucker"),("Brandon Montour","Rasmus Ristolainen")]
    dropped_days_g["Thursday"] = ["Jesper Bratt"]
    dropped_days_opp["Thursday"] = ["Steven Stamkos", "JJ Peterka", "Josh Norris"]
    dropped_days_opp["Saturday"] = ["JJ Peterka"]


In [ ]:
swapped_days_g["Monday"] = [("Sean Monahan",),("Ryan Donato",)]
dropped_days_g["Thursday"]=["Boone Jenner"]
if opp_team_id == 1:
    swapped_days_opp["Monday"] = [("Phillip Danault",),('Charlie McAvoy',)]
    dropped_days_opp["Thursday"] = ["Ivan Barbashev", "Filip Hronek"]
    dropped_days_opp["Saturday"] = ["Ivan Barbashev", "Filip Hronek"]
    swapped_days_g["Saturday"] = [
    ("Adam Fantilli", "Ryan Donato"),
    ("Zach Werenski", "Morgan Geekie"),
    ("Dmitri Voronkov", "Luke Schenn"),
    ("Boone Jenner","Erik Cernak")]
    swapped_days_g["Sunday"] = [("Vince Dunn","Jason Zucker")]
elif opp_team_id == 3:
    swapped_days_opp["Monday"] = [
        ("Tony DeAngelo",),
        ("Noah Dobson",),
        ("Dylan Holloway",),
    ]
    swapped_days_g["Saturday"] = [
    ("Adam Fantilli", "Ryan Donato"),
    ("Zach Werenski", "Morgan Geekie"),
    ("Dmitri Voronkov", "Shayne Gostisbehere"),
    ("Boone Jenner","Niko Mikkola")]
    swapped_days_g["Sunday"] = [("Vince Dunn","Jason Zucker")]
    dropped_days_opp["Thursday"] = ["Steven Stamkos", "JJ Peterka", "Josh Norris"]
    dropped_days_opp["Saturday"] = ["JJ Peterka"]


In [ ]:
# current_day = "Saturday"
current_day = None
predict_tomorrow = False

In [ ]:
teams = (g_team,opp_team)
scores = (current_g_scores,current_opp_scores)
swapped_days = (swapped_days_g,swapped_days_opp)
dropped_days = (dropped_days_g,dropped_days_opp)
current_players = (current_g_players,current_opp_players)
overall_wins,win_cats,num_players,win_differences = get_week_predictions(teams, played_players,games,skater_cats,scores,swapped_days,dropped_days,current_players,current_day=current_day,predict_tomorrow=predict_tomorrow)

print(overall_wins)
print("Win cats: ", win_cats)
print(num_players)

In [ ]:
for cat_ind,cat in enumerate(skater_cats):
    print(f"{cat:<20} {win_differences[cat_ind]}")